In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import math
from datetime import datetime,timedelta

In [ ]:
#당일 총 페이지 수
def calculator(url):

    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    number = soup.select('div.title_desc.all_my')[0].text[6:-1].strip().replace(',',"")
    maxpage = math.ceil(int(number)/10)
    maxpage_repeat = (int(maxpage)-1)*10 +1 #네이버 페이지 올라가는 식 ex 1페이지 : 1 , 2페이지 11 3페이지 21 .......
    return maxpage_repeat

In [ ]:
#네이버 기사 추출 함수
def get_navernews(n_url):
    news_content = []

    get_req = requests.get(n_url)
    get_soup = BeautifulSoup(get_req.content, 'html.parser')

    title = get_soup.select('h3#articleTitle')[0].text  
    news_content.append(title)

    get_date = get_soup.select('.t11')[0].get_text()[:11]
    news_content.append(get_date)

    _text = get_soup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    n_text = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_content.append(n_text.strip())
  
    news_content.append(n_url)
    
    newspaper = get_soup.select('#footer address')[0].a.get_text()
    news_content.append(newspaper)

    return news_content

In [ ]:
# 날짜 검색 입력
query = input("검색어를 입력하세요 : ") 
startday = input("시작날짜 입력(ex:20200617) : ")

startdatetime = datetime.strptime(startday , '%Y%m%d').date()

endday = input("마지막 날짜입력(ex:20200617) : ")

enddatetime = datetime.strptime(endday , '%Y%m%d').date()

page = 1
url = 'https://search.naver.com/search.naver?&where=news&query='+query+'&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=3&ds='+startday+'&de='+startday+'&docid=&nso=so:dd,p:from'+startday+ 'to'+ startday+ ',a:all&mynews=0&start='+str(page)+'&refresh_start=0'
maxpage_repeat = calculator(url)
news_box =[]

# 최종 날짜 까지 반복
while startdatetime < enddatetime +timedelta(days=1) :
    
    print("현재 날짜",startdatetime)
    #총페이지 까지 반복
    while page < maxpage_repeat +1 :
        print("현재페이지",page,"사이트",url)
        req = requests.get(url)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
        for urls in soup.select("._sp_each_url"):
            try :
                if urls["href"].startswith("https://news.naver.com"):
                    news_content = get_navernews(urls["href"])
                    year = news_content[1]
                    newspaper= news_content[4]
                    title=news_content[0]
                    contents=news_content[2]
                    link=news_content[3]
                    news_total = [year, newspaper, title, contents, link]
                    news_box.append(news_total)
            except Exception as e:
                print(e)
                continue
        page += 10
        url = 'https://search.naver.com/search.naver?&where=news&query='+query+'&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=3&ds='+startday+'&de='+startday+'&docid=&nso=so:dd,p:from'+startday+ 'to'+ startday+ ',a:all&mynews=0&start='+str(page)+'&refresh_start=0'

    startdatetime = startdatetime +timedelta(days=1)
    startday = str(startdatetime).replace('-','')
    page = 1
    url = 'https://search.naver.com/search.naver?&where=news&query='+query+'&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=3&ds='+startday+'&de='+startday+'&docid=&nso=so:dd,p:from'+startday+ 'to'+ startday+ ',a:all&mynews=0&start='+str(page)+'&refresh_start=0'
    maxpage_repeat = calculator(url)
    


In [ ]:
now = datetime.now() 
data = pd.DataFrame(news_box)
data.columns = ['years','newspaper','title','contents','link']
xlsx_outputFileName = "./ 검색어 {}, 기간 {} - {} 실행시간 {}-{}-{} {}시 {}분.xlsx".format(query, startday, endday,now.year, now.month, now.day, now.hour, now.minute )
data.to_excel(xlsx_outputFileName, encoding='utf-8')